In [3]:
import pandas as pd

In [2]:
df = pd.read_csv("order_brush_order.csv")
display(df)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [3]:
df["event_time"] = pd.to_datetime(df["event_time"])
display(df)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [4]:
final_list = list()
for shopid, shop_df in df.groupby("shopid"):
    shop_df = shop_df.copy()
    shop_df = shop_df.sort_values("event_time")
    shop_df["brush"] = False
    
    num_shop_orders = len(shop_df)
    for i in range(num_shop_orders):
        for j in range(i, num_shop_orders):
            window_time_diff = (shop_df["event_time"].iloc[j] - shop_df["event_time"].iloc[i]).total_seconds()
            if i > 0 and j < (num_shop_orders - 1):
                outer_window_time_diff = (shop_df["event_time"].iloc[j + 1] - shop_df["event_time"].iloc[i - 1]).total_seconds()
#             else:
#                 outer_window_time_diff = 3601

#             if window_time_diff > 3600 or outer_window_time_diff < 3600:
#                 break
            if window_time_diff <= 3600 and (i == 0 or j == (num_shop_orders - 1) or outer_window_time_diff >= 3600):
                time_df = shop_df.iloc[i:j+1]
                num_orders = len(time_df)
                num_unique_users = len(time_df["userid"].unique())
                concentration = num_orders / num_unique_users
                if concentration >= 3:
                    time_df["brush"] = True

    order_brushed_df = shop_df[shop_df["brush"]]
    if not order_brushed_df.empty:
        userids = order_brushed_df["userid"].mode().tolist()
        userids.sort()
        formatted_users = "&".join([str(userid) for userid in userids])
    else:
        formatted_users = "0"

    final_list.append([shopid, formatted_users])

C:\Users\Aloysius\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
final_df = pd.DataFrame(final_list, columns=["shopid", "userid"])

In [6]:
final_df[final_df["userid"] != "0"].head(30)

,shopid,userid
13,10159,214988798
40,10402,77819
57,10536,672345
111,42472,740844
114,42818,170385453
129,76934,190449497
180,195531,214992524
251,425364,72914921
277,599533,264511
278,605561,181682008


In [7]:
final_df.head(10)

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
5,10107,0
6,10108,0
7,10110,0
8,10132,0
9,10133,0


In [8]:
final_df.shape

(18770, 2)

In [9]:
test_df = final_df.copy()
test_df["num_userids"] = test_df["userid"].apply(lambda x: len(x.split("&")))
test_df = test_df.sort_values("num_userids", ascending = False)
test_df

,shopid,userid,num_userids
3189,51134277,29857724&212200633,2
14051,181009364,101832161&214208720,2
6456,107189025,35639374&159315857,2
9045,143281052,99517130&186080843,2
3341,54257623,1974334&107414154,2
...,...,...,...
6257,104457555,214778616,1
6256,104457372,0,1
6255,104447875,0,1
6254,104367875,0,1


In [10]:
final_df.to_csv("output.csv", index = False)